This Python-code runs the Python program 'run_polymer_simulation_once.ipynb' many times to obtain better statistics. Results-files are saved for each simulation and finally used to create an average of the quantities.

# Import of libraries 

In [1]:
import math
import numpy as np
import os

# Initialization and calculation of basic parameters

Initial parameters (fixed values for all simulation) are defined in the Initialization. Then, they are written to another python-file, 'init_parameters.ipynb'. The program 'run_polymer_simulation_once.ipynb' load this initial parameters from the file 'init_parameters.py' every time it is called. Attention: This extra file 'init_parameters.ipynb' is needed because a python program cannot load variables from this file ('main_code.ipynb') while it is run.

In [2]:
#define number of polymers segments (monomers)
Monomers = 10

#define number of bonds between segments
Bonds = Monomers - 1

#set simulation box length
simulation_box_length = 10

#set box factor (factor to multiply each box side with)
box_factor = 1

#define spring constant (Rouse model) for bonds between segments in units energy/distance^2
k = 1

#define rest bond length (same as LJ-sigma_AA)
r0 = 1

#define temperature in units of kT
kT = 1

#define drag-coefficient constant gamma for all particles
gamma = 1

#define translational diffusion constant
D_t = kT/gamma

#######################################################################

#set radius of passive particles (should be halb sigma for LJ-potential)
R_passive_partilce = 0.5

#calculate volume of passive particles
volume_passive_particle = 4/3*np.pi*(R_passive_partilce)**3

#set density of passive particles
density_passive_particles = 0.1

#calculate number of passive particles
n_passive_particles = int( (box_factor*simulation_box_length)**3 *
                          density_passive_particles / volume_passive_particle )

#######################################################################

#set radius of active particles (should be halb sigma for LJ-potential)
R_active_partilce = 0.5

#calculate volume of active particles
volume_active_particle = 4/3*np.pi*(R_active_partilce)**3

#set density of active particles
density_active_particles = 0.1

#calculate number of active particles
n_active_particles = int( (box_factor*simulation_box_length)**3 *
                         density_active_particles / volume_active_particle )

#set active force vector for active particles
force_vec_active_particle = (10, 0, 0)

#define velocity of active particles
v_ac = np.sqrt((force_vec_active_particle[0]**2+
                force_vec_active_particle[1]**2+
                force_vec_active_particle[2]**2)) #formula checked!

#set rotational diffusion constant D_r for active particles
D_r_active_particle = 0.01

#define tau_r (persistence time) for active particles
tau_r_active_particle = 1/D_r_active_particle

#######################################################################

#set epsilon and sigma for LJ-potential between all particles
#A=Monomers, B=passive crowders, C=active crowders
lj_epsilon_AA = 1
lj_sigma_AA   = r0

lj_epsilon_AB = 1
lj_sigma_AB   = 1

lj_epsilon_AC = 1
lj_sigma_AC   = 1

lj_epsilon_BB = 1
lj_sigma_BB   = 1

lj_epsilon_BC = 1
lj_sigma_BC   = 1

lj_epsilon_CC = 1
lj_sigma_CC   = 1

#define r_cut from which LJ-potential is 0 (same for all particles)
r_cut = 2**(1/6)*lj_sigma_AA
#######################################################################

#define integration step time
dt = 10**(-4)

#define number of integration steps: two magnitudes more than rotational diff. constant
integration_steps = 1/dt*tau_r_active_particle*10

#define number of points for representation
number_of_points = 10000

#calculate return period to log data to file
return_period = integration_steps/number_of_points

Create file 'initial_values.py' and write the set parameters into it; this file is called every time when the simulation 'run_polymer_simulation_once.py' starts.

In [3]:
#safe initial parameters
init_parameters = (

    "Monomers = "                  +str(Monomers)+"\n"                   +
    "Bonds = "                     +str(Bonds)+"\n"                      +
    "simulation_box_length ="      +str(simulation_box_length)+"\n"      +
    "box_factor = "                +str(box_factor)+"\n"                 +
    "k = "                         +str(k)+"\n"                          +
    "r0 = "                        +str(r0)+"\n"                         +
    "kT = "                        +str(kT)+"\n"                         +
    "gamma = "                     +str(gamma)+"\n"                      +
    "D_t = "                       +str(D_t)+"\n\n"                      +
    
    "R_passive_partilce = "        +str(R_passive_partilce)+"\n"         +
    "volume_passive_particle = "   +str(volume_passive_particle)+"\n"    +
    "density_passive_particles = " +str(density_passive_particles)+"\n"  +
    "n_passive_particles = "       +str(n_passive_particles)+"\n\n"      +
    
    "R_active_partilce = "         +str(R_active_partilce)+"\n"          +
    "volume_active_particle = "    +str(volume_active_particle)+"\n"     +
    "density_active_particles = "  +str(density_active_particles)+"\n"   +
    "n_active_particles = "        +str(n_active_particles)+"\n"         +
    "force_vec_active_particle = " +str(force_vec_active_particle)+"\n"  +
    "v_ac = "                      +str(v_ac)+"\n"                       +
    "D_r_active_particle = "       +str(D_r_active_particle)+"\n"        +
    "tau_r_active_particle = "     +str(tau_r_active_particle)+"\n\n"    +
  
    "lj_epsilon_AA = "             +str(lj_epsilon_AA)+"\n"              +
    "lj_sigma_AA = "               +str(lj_sigma_AA)+"\n"                +
    "lj_epsilon_AB = "             +str(lj_epsilon_AB)+"\n"              +
    "lj_sigma_AB = "               +str(lj_sigma_AB)+"\n"                +
    "lj_epsilon_AC = "             +str(lj_epsilon_AC)+"\n"              +
    "lj_sigma_AC = "               +str(lj_sigma_AC)+"\n"                +
    "lj_epsilon_BB = "             +str(lj_epsilon_BB)+"\n"              +
    "lj_sigma_BB = "               +str(lj_sigma_BB)+"\n"                +
    "lj_epsilon_BC = "             +str(lj_epsilon_BC)+"\n"              +
    "lj_sigma_BC = "               +str(lj_sigma_BC)+"\n"                +
    "lj_epsilon_CC = "             +str(lj_epsilon_CC)+"\n"              +
    "lj_sigma_CC = "               +str(lj_sigma_CC)+"\n"                +
    "r_cut = "                     +str(r_cut)+"\n\n"                    +
    
    "dt = "                        +str(dt)+"\n"                         +
    "integration_steps = "         +str(integration_steps)+"\n"          +
    "number_of_points = "          +str(number_of_points)+"\n"           +
    "return_period = "             +str(return_period)  )

#make py-file (same for all simulations with same parameters) if not already exists
file_init_parameters = open('initial_parameters'+
                            '.py', 'w')

file_init_parameters.write(init_parameters)

file_init_parameters.close()

# Run the entire simulation once

In [4]:
#%run run_polymer_simulation_once.ipynb